# Imports

In [ ]:
import pandas as pd

# Data Collection

In [ ]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_X_ec.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_Y_ec.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/test_X_ec.csv

--2021-09-16 12:48:09--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_X_ec.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.62.52
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.62.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27774 (27K) [text/csv]
Saving to: ‘train_X_ec.csv’

train_X_ec.csv      100%[===================>]  27.12K   109KB/s    in 0.2s    

2021-09-16 12:48:11 (109 KB/s) - ‘train_X_ec.csv’ saved [27774/27774]

--2021-09-16 12:48:11--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_Y_ec.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3

In [ ]:
train_X_df = pd.read_csv('train_X_ec.csv')
train_Y_df = pd.read_csv('train_Y_ec.csv', header=None).squeeze()
test_X_df = pd.read_csv('test_X_ec.csv')

# Understanding the data

In [ ]:
train_X_df

,Time Spent on Website,Duration of Membership,Time Spent on App,Session Duration\t
0,69.156561,20.457356,32.219553,62.186251
1,67.975829,19.787373,30.887689,57.816544
2,66.702558,18.986008,32.711838,58.286399
3,65.371799,19.474788,29.409797,62.118480
4,66.551924,18.933956,32.462605,59.442156
...,...,...,...,...
369,68.259847,18.797766,31.374397,60.997459
370,66.542426,19.151099,31.890124,62.672003
371,65.098581,18.278514,32.896052,59.292303
372,64.446483,20.232239,32.636058,60.393138


In [ ]:
train_X_df.dtypes

Time Spent on Website     float64
Duration of Membership    float64
Time Spent on App         float64
Session Duration\t        float64
dtype: object

In [ ]:
train_X_df.isna().sum()

Time Spent on Website     0
Duration of Membership    0
Time Spent on App         0
Session Duration\t        0
dtype: int64

# Implementation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression

pipe = Pipeline(steps=[('scaler', RobustScaler()),
                       ('classifier', LinearRegression())])

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV


param_distributions = {
    'scaler': [ StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
    'classifier': [LinearRegression(), Ridge(), Lasso()]
}

random_search_cv = RandomizedSearchCV(pipe, param_distributions=param_distributions, n_iter=20, scoring='neg_mean_squared_error', refit=True, cv=5, random_state=42) 
random_search_cv.fit(train_X_df, train_Y_df)
print(random_search_cv.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'classifier': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)}


In [ ]:
random_search_cv.score(train_X_df, train_Y_df)

-214.77131654928323

In [ ]:
best_model = random_search_cv.best_estimator_
predicted_test_Y = best_model.predict(test_X_df)

Writing the predicted values to CSV file

In [ ]:
pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_ec.csv', header=None, index=False)